#### 1. Implementación de la red neuronal

In [ ]:
# Modelo mejorado con arquitectura más profunda y técnicas avanzadas
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()

if K.image_data_format() == 'channels_last':
    # (width, height, channels)
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_data_format() == 'channels_first':
    # (channels, width, height)
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')

# Primera capa convolucional
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(BatchNormalization())  # Normalización por lotes para estabilizar el entrenamiento
model.add(Activation('relu'))

# Segunda capa convolucional
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Tercera capa convolucional
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Cuarta capa convolucional adicional para mayor capacidad
model.add(Convolution2D(128, (3, 3), strides=(1, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Aplanar para capas densas
model.add(Flatten())

# Primera capa densa
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))  # Dropout para evitar sobreajuste

# Segunda capa densa
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Capa de salida
model.add(Dense(nb_actions))
model.add(Activation('linear'))

print(model.summary())

#### 2. Implementación de la solución DQN con Prioritized Experience Replay

In [ ]:
# Configuración de la memoria con Prioritized Experience Replay
memory = PrioritizedMemory(limit=100000,  # Memoria grande para almacenar más experiencias
                          alpha=0.6,      # Prioridad basada en errores TD
                          beta=0.4,       # Corrección de importancia del muestreo
                          beta_increment=0.0005,  # Incremento gradual de beta
                          window_length=WINDOW_LENGTH)

# Procesador para las observaciones
processor = AtariProcessor()

# Política de exploración con decaimiento lineal
# Comenzamos con exploración completa (1.0) y reducimos gradualmente a 0.05
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1.0, value_min=0.05, value_test=0.01,
                              nb_steps=150000)  # Decaimiento más lento para mejor exploración

# Definición del agente DQN
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy,
               memory=memory, processor=processor,
               nb_steps_warmup=10000,  # Más pasos de calentamiento para llenar la memoria
               gamma=0.99,  # Factor de descuento alto para valorar recompensas futuras
               target_model_update=10000,  # Actualización menos frecuente de la red objetivo
               train_interval=4,  # Entrenar cada 4 pasos para estabilidad
               delta_clip=1.0)  # Recortar el error delta para evitar explosiones de gradiente

# Compilación del agente con optimizador RMSprop (mejor para DQN en Atari)
dqn.compile(RMSprop(learning_rate=0.00025, rho=0.95, epsilon=0.01), metrics=['mae'])

#### Configuración de callbacks y entrenamiento

In [ ]:
# Configuración de callbacks
weights_filename = 'dqn_ultimate_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_ultimate_' + env_name + '_weights_episode_{episode}.h5f'
log_filename = 'dqn_ultimate_{}_log.json'.format(env_name)
visualization_log = 'dqn_ultimate_{}_visualization.json'.format(env_name)

# Crear directorio para checkpoints si no existe
checkpoint_dir = 'checkpoints'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Callbacks personalizados
callbacks = [
    # Guardar pesos cada 5 episodios
    EpisodeCheckpoint(os.path.join(checkpoint_dir, checkpoint_weights_filename), interval=5),
    
    # Visualizar progreso cada 5 episodios
    TrainingVisualization(visualization_log, plot_interval=5),
    
    # Ajustar tasa de aprendizaje cada 50 episodios
    LearningRateScheduler(initial_lr=0.00025, min_lr=0.00001, decay_factor=0.5, decay_episodes=50),
    
    # Logger estándar
    FileLogger(log_filename, interval=100)
]

In [ ]:
# Entrenamiento del agente
dqn.fit(env, callbacks=callbacks, nb_steps=250000, log_interval=1000, visualize=False)

# Guardar pesos finales
dqn.save_weights(weights_filename, overwrite=True)

#### Visualización de resultados del entrenamiento

In [ ]:
# Cargar y visualizar datos de entrenamiento
if os.path.exists(visualization_log):
    with open(visualization_log, 'r') as f:
        data = json.load(f)
    
    plt.figure(figsize=(15, 10))
    
    # Gráfico de recompensas
    plt.subplot(2, 2, 1)
    plt.plot(data['episode_rewards'])
    plt.title('Recompensas por episodio')
    plt.xlabel('Episodio')
    plt.ylabel('Recompensa')
    
    # Gráfico de promedio móvil de recompensas
    plt.subplot(2, 2, 2)
    plt.plot(data['moving_avg_rewards'])
    plt.title('Promedio móvil de recompensas')
    plt.xlabel('Episodio')
    plt.ylabel('Recompensa promedio')
    
    # Gráfico de pérdidas
    plt.subplot(2, 2, 3)
    plt.plot(data['episode_losses'])
    plt.title('Pérdida por episodio')
    plt.xlabel('Episodio')
    plt.ylabel('Pérdida')
    
    # Gráfico de MAE
    plt.subplot(2, 2, 4)
    plt.plot(data['episode_maes'])
    plt.title('MAE por episodio')
    plt.xlabel('Episodio')
    plt.ylabel('MAE')
    
    plt.tight_layout()
    plt.show()

#### Test del agente entrenado

In [ ]:
# Probar con los mejores pesos
best_weights_filename = os.path.join(checkpoint_dir, 'dqn_ultimate_' + env_name + '_weights_episode_best.h5f')
if os.path.exists(best_weights_filename):
    print(f"Cargando los mejores pesos desde: {best_weights_filename}")
    dqn.load_weights(best_weights_filename)
else:
    print(f"No se encontraron los mejores pesos, usando los pesos finales: {weights_filename}")
    dqn.load_weights(weights_filename)

# Test de n episodios para calcular la recompensa final
dqn.test(env, nb_episodes=10, visualize=True)

#### 3. Justificación de los parámetros seleccionados y de los resultados obtenidos

### Justificación de los parámetros seleccionados

En esta implementación optimizada para máximo rendimiento, se han incorporado numerosas mejoras basadas en investigaciones recientes en aprendizaje por refuerzo profundo:

1. **Prioritized Experience Replay (PER)**:
   - Implementación completa de PER que prioriza experiencias con mayor error TD.
   - Parámetros α=0.6 y β=0.4 inicialmente, con incremento gradual de β hasta 1.0.
   - Esta técnica permite un aprendizaje más eficiente al muestrear con mayor frecuencia experiencias más informativas.

2. **Arquitectura de red neuronal mejorada**:
   - Red convolucional profunda con 4 capas convolucionales (vs 3 en implementaciones estándar).
   - Capa adicional de 128 filtros para capturar patrones más complejos.
   - BatchNormalization después de cada capa para estabilizar y acelerar el entrenamiento.
   - Dos capas densas (512 y 256 neuronas) para mayor capacidad de representación.
   - Dropout (20%) para prevenir el sobreajuste.

3. **Memoria de experiencia mucho más grande**:
   - Aumentada a 100,000 experiencias para mantener un historial más amplio.
   - Permite al agente aprender de una variedad mucho mayor de situaciones.

4. **Exploración más efectiva**:
   - Decaimiento de epsilon más lento (150,000 pasos).
   - Valor mínimo de epsilon reducido a 0.05 para mantener algo de exploración incluso al final.
   - Valor de test reducido a 0.01 para evaluación más determinista.

5. **Entrenamiento extenso**:
   - 250,000 pasos de entrenamiento para permitir un aprendizaje profundo.
   - 10,000 pasos de calentamiento para llenar adecuadamente la memoria antes de comenzar el aprendizaje.

6. **Callbacks personalizados**:
   - EpisodeCheckpoint: Guarda pesos cada 5 episodios y mantiene los mejores pesos basados en recompensa.
   - TrainingVisualization: Visualiza el progreso del entrenamiento con gráficos detallados.
   - LearningRateScheduler: Ajusta automáticamente la tasa de aprendizaje para evitar estancamiento.

7. **Optimizador RMSprop optimizado**:
   - Parámetros específicos (learning_rate=0.00025, rho=0.95) que han demostrado mejor rendimiento en DQN para juegos Atari.

### Resultados esperados

Con estas mejoras, esperamos que el agente logre una puntuación significativamente superior al requisito mínimo de 20 puntos. Basándonos en implementaciones similares en la literatura científica, este modelo optimizado debería ser capaz de alcanzar puntuaciones en el rango de 500-1500 puntos en Space Invaders, lo que representa un rendimiento muy competente.

Las mejoras implementadas siguen las mejores prácticas establecidas en los artículos seminales sobre DQN y sus variantes, particularmente:

1. **Prioritized Experience Replay** (Schaul et al., 2015)
2. **Deep Q-Network con mejoras arquitectónicas** (Mnih et al., 2015)
3. **Técnicas de estabilización del entrenamiento** como BatchNormalization y Dropout
4. **Ajuste dinámico de hiperparámetros** durante el entrenamiento

La combinación de estas técnicas avanzadas debería permitir al agente desarrollar estrategias sofisticadas para maximizar su puntuación en Space Invaders, superando ampliamente los requisitos mínimos del proyecto.